In [1]:
# Import OpenAI package into Python:
import openai
# Import Azure OpenAI package into Python:
from openai import AzureOpenAI
# Import environment packages:
from dotenv import load_dotenv
import os
# Import GUI package:
import panel as pn




ModuleNotFoundError: No module named 'xyzservices'

In [ ]:
# Load .env file and get API key, API base and deployment name:
load_dotenv()  
api_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    azure_endpoint = api_base,
    api_key=api_key,
    api_version=api_version
)

# Load single PDF:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("data/barbie.pdf")

# Split content into chunks.
# Import a splitter module called RecursiveCharacterTextSplitter:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Instantiate the splitter:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# Do the splitting:
splits = text_splitter.split_documents(loader.load())

# Embed and store splits in Chroma database.
# Import the Chroma and embeddings modules:
from langchain.vectorstores import Chroma
# from langchain.embeddings import OpenAIEmbeddings
# Instantiate the Chroma vector store with arguments:
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding_model
)
# Instantiate a retriever based on the vector store:
retriever = vectorstore.as_retriever()

# Instantiate a prompt.
# We will use a standard RAG prompt downloaded from Langsmith:
# https://smith.langchain.com/hub/rlm/rag-prompt
from langchain import hub
rag_prompt = hub.pull("rlm/rag-prompt")

llm = AzureOpenAI(
    model="gpt-4o-mini",
    deployment_name="gpt-4o-mini",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
    temperature=0
)

# Instantiate RAG chain:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)

# Get completion:
# REPLACE '_____________________' WITH THE NEW QUERY
# completion = rag_chain.invoke("_____________________")
completion = rag_chain.invoke("Who are the stars of the 2023 Barbie movie?")
print(completion.content)
